In [1]:
import ufl
from ufl import  TrialFunction, TestFunction, FacetNormal, lhs, rhs,SpatialCoordinate

import dolfinx
from dolfinx import mesh, fem, io
from dolfinx.fem import FunctionSpace, VectorFunctionSpace, locate_dofs_geometrical, form
from dolfinx.fem.petsc import LinearProblem

from mpi4py import MPI
import petsc4py.PETSc
import numpy as np
import matplotlib.pyplot as plt


# Operators:
class Infix:

    def __init__(self, function):
        self.function = function

    def __ror__(self, other):
        return Infix(lambda x, self=self, other=other: self.function(other, x))

    def __or__(self, other):
        return self.function(other)

    def __call__(self, value1, value2):
        return self.function(value1, value2)


dot = Infix(ufl.dot)
inner = Infix(ufl.inner)

def vector(*args):
    return ufl.as_vector(tuple(args))

from ufl import exp,sym,tr,sqrt
from ufl import nabla_div as div
from ufl import nabla_grad as grad
from ufl import as_matrix as matrix

npor = Infix(np.logical_or)
npand = Infix(np.logical_and)

def I(func_like):
    """Create matrix Identity dimension of func_like

    Args:
        func_like (Function): Give geometric dimension

    Returns:
        Tensor: Identity
    """
    return ufl.Identity(func_like.geometric_dimension())

# Classes:


def Function(space, func=None):
    """Function on new space. Default create just Function on space

    Args:
        space (FunctionSpace): New space
        function (): Ufl_function, fem.Function, callable function, fem.Constant, constant

    Returns:
        fem.Function: Function
    """
    if func == None: return fem.Function(space)

    result = fem.Function(space)

    if not callable(func):
        expression = lambda x: x[0] * 0 + func

    elif hasattr(func, 'function_space'):
        expression = func

    elif hasattr(func, 'value'):
        expression = lambda x: x[0] * 0 + func.value

    elif hasattr(func, 'ufl_domain'):
        expression = fem.Expression(func, space.element.interpolation_points())
    else:
        expression = func

    result.interpolate(expression)
    return result


def Constant(space, const):
    """Constant on space

    Args:
        space (fem.FunctionSpace): Space
        const (auny number): Any number

    Returns:
        fem.function.Constant: Constant on space
    """
    return fem.Constant(space, petsc4py.PETSc.ScalarType(const))


def create_facet_markers(domain, bound_markers):
    """Mark facets under conditious

    Args:
        domain (Domain): Domain
        bound_markers (mark,python_function): List of mark and function
    
    Return:
        tags(mesh.meshtags): Marked facets
    """
    facet_indices, facet_markers = [], []
    for (marker, condition) in bound_markers:
        facets = mesh.locate_entities(
            domain,
            domain.topology.dim - 1,
            condition,
            )
        facet_indices.append(facets)
        facet_markers.append(np.full_like(facets, marker))
    facet_indices = np.hstack(facet_indices).astype(np.int32)
    facet_markers = np.hstack(facet_markers).astype(np.int32)
    sorted_facets = np.argsort(facet_indices)
    facet_tags = mesh.meshtags(
        domain,
        domain.topology.dim - 1,
        facet_indices[sorted_facets],
        facet_markers[sorted_facets],
        )

    return facet_tags


def create_connectivity(domain):
    """Need to compute facets to Boundary value

    Args:
        domain (Mesh): Domain
    """
    domain.topology.create_connectivity(
        domain.topology.dim - 1,
        domain.topology.dim,
        )


def DirichletBC(space, func, combined_marker):
    """Create Dirichlet condition

    Args:
        space (fem.FunctionSpace): Function space
        func (fem.function): fem.Function
        combined_marker (Any): One from next
        \nFunction - boundary marker function
        \nAll - all boundary
        \n(mesh.meshtags, marker) - list or tuple, marker of boundary from Marked_facets - mesh.meshtags
        
    Returns:
        condition (dirichletbc): Dirichlet condition
    """

    def get_exterior_facets():
        exterior_facets = mesh.exterior_facet_indices(domain.topology)
        return exterior_facets

    def all_dirichlet(dofs, func):
        if hasattr(func, 'function_space'):
            bc = fem.dirichletbc(dofs=dofs, value=func)
        else:
            bc = fem.dirichletbc(V=space, dofs=dofs, value=func)
        return bc

    domain = space.mesh

    if combined_marker == 'All':
        facets = get_exterior_facets()
        dofs = fem.locate_dofs_topological(
            space,
            domain.topology.dim - 1,
            facets,
            )

    elif type(combined_marker) == (tuple or list):
        marked_facets, marker = combined_marker
        facets = marked_facets.find(marker)
        dofs = fem.locate_dofs_topological(
            space,
            domain.topology.dim - 1,
            facets,
            )
    else:
        dofs = fem.locate_dofs_geometrical(space, marker=combined_marker)

    bc = all_dirichlet(dofs, func)

    return bc


# Post processing:
def errors_L(uS, uEx):
    """Compute error norm on boundary

    Args:
        uS (Function): Numeric solution
        uEx (Function): Exact or model solution

    Returns:
        List: L1 and L2 norms
    """
    domain = uS.function_space.mesh

    V_err = FunctionSpace(
        domain,
        ('CG', uS.function_space.element.space_dimension),
        )
    uEx = Function(V_err, uEx)
    L1_scalar = fem.assemble_scalar(form((uS-uEx) * dx))
    L2_scalar = fem.assemble_scalar(form((uS - uEx)**2 * dx))

    L1_err = np.abs(domain.comm.allreduce(L1_scalar, op=MPI.SUM))
    L2_err = np.sqrt(domain.comm.allreduce(L2_scalar, op=MPI.SUM))
    return (L1_err, L2_err)


def line_collision(domain, line_cord):
    """Generate points and cells of colliding domain and line

    Args:
        domain (mesh): Domain
        line_cord (array): 3D line contervertor of coordinates 

    Returns:
        Tuple: Collision points and collision cells
    """
    bb_tree = dolfinx.geometry.BoundingBoxTree(domain, domain.topology.dim)

    cells_on_line = []
    points_on_line = []
    cell_candidates = dolfinx.geometry.compute_collisions(bb_tree, line_cord.T)
    colliding_cells = dolfinx.geometry.compute_colliding_cells(
        domain, cell_candidates, line_cord.T
        )
    for i, point in enumerate(line_cord.T):
        if len(colliding_cells.links(i)) > 0:
            points_on_line.append(point)
            cells_on_line.append(colliding_cells.links(i)[0])

    points_on_line = np.array(points_on_line, dtype=np.float64)

    return (points_on_line, cells_on_line)


def graph2D(fig, lists, natural_show=False, points_on=False):
    """Create graph from fem.Function

    Args:
        fig (plt.Figure): Figure
        lists (fem.Function , plt.Axes, str): List of (u, curent axes, title)
        method (bool): Graph method True = tripcolor, False = tricontourf
    """

    def data_construct(dofs, x_array):
        data = np.column_stack((dofs[:, 0:2], x_array))
        x_data = data[:, 0]
        y_data = data[:, 1]
        z_data = data[:, 2]
        return [x_data, y_data, z_data]

    for list in lists:
        fig, ax = plt.subplots()
        plt.close()
        u, ax, title = list
        dofs = u.function_space.tabulate_dof_coordinates()
        ax.set_title(title)
        data = data_construct(dofs, u.x.array)

        if points_on: ax.plot(data[0], data[1], 'o', markersize=2, color='grey')

        if natural_show:
            plot = ax.tripcolor(*data)
        else:
            try:
                levels = np.linspace(u.x.array.min(), u.x.array.max(), 10)
                plot = ax.tricontourf(
                    *data,
                    levels=levels,
                    )
            except:
                print(f'{title} - error')

        ax.set_aspect(1)
        fig.colorbar(plot, ax=ax)
    return

In [2]:
L = 1
W = 0.2
domain = mesh.create_box(
    MPI.COMM_WORLD,
    [np.array([0, 0, 0]), np.array([L, W, W])],
    [10, 3, 3],
    cell_type=mesh.CellType.hexahedron,
    )
create_connectivity(domain=domain)
V = VectorFunctionSpace(domain, ('CG', 1))

u, v = TrialFunction(V), TestFunction(V)
x, y, z = SpatialCoordinate(V)
ds = ufl.Measure("ds", domain=domain)
dx = ufl.Measure('cell', subdomain_id='everywhere')

delta = 0.4 * (W / L)**2
lamb = 1.25
mu = 1
rho = 2
eT = vector(0, 0.01, -0.02)
euD1 = vector(0, 0, 0)
euD2 = vector(0, 0, -0.2)
ef = vector(0, 0, -rho * delta)
epsilon = lambda u: sym(grad(u))
sigma = lambda u: lamb * div(u) * I(u) + 2 * mu * epsilon(u)

T = Constant(V, eT)
uD1 = Constant(V, (0, 0, 0))
uD2 = Constant(V, euD2)
f = Constant(V, ef)

bcs = [
    DirichletBC(V, uD1, lambda x: np.isclose(x[0], 0)),
    DirichletBC(V, uD2, lambda x: np.isclose(x[0], 1)),
    ]


F = (sigma(u)|inner|epsilon(v)) * dx - (f|dot|v) * dx - (T|dot|v) * ds

In [3]:
problem = LinearProblem(
    lhs(F),
    rhs(F),
    bcs=bcs,
    petsc_options={
        "ksp_type": "preonly", "pc_type": "lu"
        },
    )
uS = problem.solve()

In [4]:
s = sigma(uS) - 1 / 3 * tr(sigma(uS)) * I(uS)
estress = sqrt(3 / 2 * s**2)

V_von = FunctionSpace(domain, ('CG', 1))
stress = Function(V_von, estress)

In [5]:
with io.VTKFile (
    domain.comm,
    '/home/VTK/Elasticity_files/elasticity.pvd',
    'w'
) as file:
    # file.write_mesh(domain)
    uS.name = 'Deformation'
    stress.name = 'Stress von Mises'
    # Tf.name = 'External power'
    file.write_function([uS,stress])

# Vector function problem

In [6]:
    cord = SpatialCoordinate(V)
    if len(cord) == 1:
        x_cargo = cord[0] - cord[0]
    else:
        x_cargo = vector(*cord) - vector(*cord)

In [7]:
print(
Function(
    V,
    vector(*f.value)-vector(*f.value)+ x_cargo,
    ).x.array
)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [8]:
vector(*f.value)

ListTensor(Zero((), (), ()), Zero((), (), ()), FloatValue(-0.03200000000000001))

In [15]:
def Function1(space, func=None):
    """Function on new space. Without func just create a Function on space. 
    \nScalars - UFL_function, fem.Function, callable function, fem.Constant, constant
    \nVectors - UFL_vector, callable function which returns list of elements,
    Args:
        space (FunctionSpace): New space
        function (): 

    Returns:
        fem.Function: Function
    """

    result = fem.Function(space)
    tupe = str(func.__class__)[8:-2]
    cord = SpatialCoordinate(space)
    cargo_vector = vector(*map(lambda x, y: x - y, cord, cord))

    # None
    if func == None:
        return result

    # fem.Function
    elif tupe == ('dolfinx.fem.function.Function'):
        expression = func

    # fem.Constant
    elif tupe == ('dolfinx.fem.function.Constant'):
        if len(func.ufl_shape) == 0 : func2 = func.value + (cord[0] - cord[0])
        else: func2 = vector(*func.value) + cargo_vector
        expression = fem.Expression(func2, space.element.interpolation_points())

    # ufl object
    elif tupe[:3] == 'ufl':
        if len(func.ufl_shape) !=0: func2 = func + cargo_vector
        else: func2 = func
        expression = fem.Expression(func2, space.element.interpolation_points())

    # Python function
    elif tupe == ('function'): expression = func
 
    # Number
    elif not hasattr(func, '__call__'):
        if hasattr(func,'__getitem__'): func2 =  vector(*func) + cargo_vector
        else: func2 = func + (cord[0] - cord[0])
        expression = fem.Expression(func2, space.element.interpolation_points())


    result.interpolate(expression)
    return result

In [10]:
a = [1,2,3]
b = [1,2,3]
print(*map(lambda x,y: x-y,a,b))

0 0 0


In [11]:
Function(
    V,
    vector(*T.value) + vector(
        *map(
            lambda x, y: x - y,
            SpatialCoordinate(V),
            SpatialCoordinate(V),
            )
        ),
    ).x.array;

In [12]:
f3_compar = Function(V, vector(x - x, 0, 0))
f3_compar.name = 'f3 comp'

f6_func = lambda x: (x[0], x[1], x[0] * 0+1)

f1 = Function1(V, uS)
f2 = Function1(V, T)
f3 = Function1(V, vector(0, 0, 0))
f4 = Function1(V, vector(1, 2, 0))
f5 = Function1(V, vector(x, y, 1))
f6 = Function1(V, f6_func)
f7 = Function1(V, (1, 3, 4))

f_list = [f1, f2, f3, f4, f5, f6, f7]
count = 1
for i in f_list:
    i.name = f'f{count}'
    count += 1

In [16]:
const = Constant(V_von, 6)
t6_func = lambda x: x[0] + x[1]

t1 = Function1(V_von, stress)
t2 = Function1(V_von, const)
t3 = Function1(V_von, 0)
t4 = Function1(V_von, x)
t5 = Function1(V_von, x + y)
t6 = Function1(V_von, t6_func)
t7 = Function1(V_von, t6 - t5)

t_list = [t1, t2, t3, t4, t5, t6, t7]
count = 1
for i in t_list:
    i.name = f't{count}'
    count += 1

In [17]:
import shutil
try:
    shutil.rmtree('/home/VTK/Elasticity_files')
except:
    print('Directory empty yet')

In [18]:
with io.VTKFile (
    domain.comm,
    '/home/VTK/Elasticity_files/test.pvd',
    'w'
) as file:
    file.write_function([f3_compar,*f_list,*t_list])

In [ ]:
a0 = x+y
a =vector(0,0,0)
b = vector(1,0,0)
c= vector(x,2,y)
d=vector(x-x,0,0)
print(type(a),type(b),type(c),type(d),type(a0))

<class 'ufl.constantvalue.Zero'> <class 'ufl.tensors.ListTensor'> <class 'ufl.tensors.ListTensor'> <class 'ufl.tensors.ListTensor'> <class 'ufl.algebra.Sum'>


In [ ]:
a0.ufl_shape

()

In [ ]:
print(a,'\n',b,'\n',c,'\n',d)

0 (shape (3,)) 
 [1, 0, 0] 
 [x[0], 2, x[1]] 
 [x[0] + -1 * x[0], 0, 0]


In [ ]:
# fem.Expression(a+d, V.element.interpolation_points())
f1 = Function1(V, b+d)
f1.x.array

array([1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1.,
       0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0.,
       1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1.,
       0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0.,
       1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1.,
       0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0.,
       1., 0., 0., 1., 0., 0., 1., 0., 0.])

In [ ]:
e1 = Constant(V,(0.,0,0)).value
print(e1)
e2= vector(*e1)


[4.68880007e-310 6.92031841e-310 1.58101007e-322]


# Problem with div grad

In [ ]:
testf2 = Function(V, div(sym(grad(uS))))
testf2.x.array

array([ 0.13520928, -0.30563969,  1.07687124, -0.1933472 ,  0.15846901,
       -0.26971689,  0.19162666, -0.2967705 ,  1.07816788, -0.33369451,
        0.1892422 , -0.25361774, -0.11085528, -0.30434304,  1.07059534,
        0.20992064,  0.16693495, -0.27592499, -0.05613061, -0.30434304,
        1.07816788,  0.07140286,  0.17244426, -0.25574164,  0.20465949,
       -0.2967705 ,  1.07059534, -0.3963923 ,  0.1892422 , -0.27592499,
       -0.04034312, -0.30434304,  1.07687124,  0.0046927 ,  0.17244426,
       -0.26971689, -0.20465949, -0.2967705 ,  1.07059534,  0.3963923 ,
        0.1892422 , -0.27592499, -0.15099677, -0.30563969,  1.07816788,
        0.26005736,  0.15846901, -0.25574164, -0.03776398,  0.24202746,
       -0.50166298, -0.03892628,  0.18147657, -0.52083004,  0.00955219,
        0.22761762, -0.47468899,  0.0107145 ,  0.22510218, -0.51858826,
       -0.13520928, -0.30563969,  1.07687124,  0.1933472 ,  0.15846901,
       -0.26971689, -0.03776398,  0.18147657, -0.47468899,  0.00

In [ ]:
testf3 = Function(V, div(grad(uS)))
testf3.x.array

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
type(sym(grad(uS)))


ufl.tensoralgebra.Sym

In [ ]:
type(grad(uS))

ufl.differentiation.NablaGrad